<a href="https://colab.research.google.com/github/torresmateo/hierarchical-clustering-lab/blob/master/Hierarchical_Clustering_Lab.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Agglomerative Clustering in Python and scikit-learn

(You don't need to setup a python environment yourself for this lab, simply click the "Open in Colab" button and you will get a working python environment)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering

# Part 1 - Swiss Roll dataset 
To begin, we generate a small dataset with 1500 points, that are distributed in a spiral, it's commonly known as a "swiss roll dataset".
Here we will use 3 dimensions to be able to visualise the data, but remember that in real datasets the dimensionality is usually way higher.

In [ ]:
from sklearn.datasets import make_swiss_roll
# Generate data (swiss roll dataset)
n_samples = 1500
noise = 0.05
X, _ = make_swiss_roll(n_samples, noise=noise)
# Make it thinner
X[:, 1] *= .5

let's look at the structure of the dataset. With the `shape` property, we can see the dimensionality of the data (as expected, 1500 datapoints in 3 dimensions)

In [ ]:
X.shape

Now, let's plot the dataset in 3D. You can see how the dataset is distributed in a spiral.

In [ ]:
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(10, -70)
ax.scatter(X[:, 0], X[:, 1], X[:, 2],
           color='r',
           s=20, edgecolor='k')
plt.title('Swiss Roll Dataset')
plt.show()
plt.close('all')

## Agglomerative Clustering

To attempt and describe the dataset, we'll use a hierarchical clustering, implemented through the [`AgglomerativeClustering`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) class from sciki-learn, and you'll see how different types of linkages structure the same data differently.

### Excercise 1.

In the cell below, I provide example code to run the hierarchical clustering $(n=6)$ on our swiss roll dataset, using a single linkage, meaning that datapoints will be included in the closest cluster, as determined by a distance. By default, scikit-learn is configured with the euclidean distance (see the `affinity` parameter in the documentation). I encourage you to copy the code below into a new cell and see how other affinities produce different results.

You will see that this linkage does not capture the structure of the data very well. Ideally, we would like to see clusters coming one after the other in the "ribbon" of datapoints that is rolled into a spiral.

Your task for this exercise is to change the `linkage` parameter from `single` to a linkage that instead of minimising the **distance**, minimises the **variance** of the clusters being merged.

I also provide a utility function: `plot_clusters`, that you can use throughout the lab to visualise your clusters.

In [ ]:
# this is a utility function that you can use to visualise the generated clusters.
def plot_clusters(labels):
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.view_init(10, -70)
    for l in np.unique(labels):
        ax.scatter(X[labels == l, 0], 
                   X[labels == l, 1], 
                   X[labels == l, 2],
                   color=plt.cm.jet(np.float(l) / np.max(labels + 1)),
                   s=20, edgecolor='k')
    plt.title('Swiss Roll Dataset')
    plt.show()
    plt.close('all')

# in a single line, we create the AgglomerativeClustering using a single linkage, 
# then we fit it to using our generated datapoints
single_linkage = AgglomerativeClustering(n_clusters=6, linkage='single').fit(X)
# we extract the labels (assgined clusters) corresponding to each of the datapoints
single_linkage_labels = single_linkage.labels_

# visualise the clusters
plot_clusters(single_linkage_labels)

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined in the previous cell 
# to visualise your results, you don't have to redefine it.

You should have seen better results with the new linkage option. However, you can probably tell that the clustering is still far from our ideal outcome. With any of the options in the `linkage` parameter, we are getting clusters that don't follow the ribbon, and instead assing to the same cluster datapoints that lie far away from each other if you were to unroll the spiral flat.

To achieve this, we can add a connectivity matrix that will provide and go from an unstructured hierarchical clustering to a structured one.

### Excersice 2

In this case, your task will be to set the `connectivity` parameter of the `AgglomerativeClustering`. This time, however, you don't have all the options listed in the documentation, since you can set this parameter to an arbitraty connectivity matrix, or a function that will transform the data and produce the connectivity matrix itself.

You can start by reading the documentation of [`RadiusNeighborsTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsTransformer.html), a class that transforms a dataset into a weighted graph of neighbours nearest than a radius. An alternative to this, is to use the k-nearest neighbours class [`KNeighborsTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsTransformer.html). Using these classes, you would extract a connectivity matrix (the weighted graph between the datapoints), and use that graph as to set the `connectivity` parameter of `AgglomerativeClustering`.

Tips: 
* scikit-learn provides helper functions for several classes that implement common transformations to datasets. Read the linked documentation an d try to find these helper functions to simplify your solution.
* Remember to import the classes and methods that you want to use.

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined above
# to visualise your results, you don't have to redefine it.

# Part 2 - Compare Linkages
In the first part of the lab, you've seen that the linkages provide different ways to interpret the structure of the dataset. Let's explore this with more detail in 2D. In this part, you should identify which linkages are better for a different structures of data in 2D.

As I did in Part 1, I provide a function to plot the clusters. This time, you have to provide the title of the plot. I encourage you to change the parameters that generate the toy datasets, and explore the [datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) documentation to if you want to explore with further datasets provided by scikit-learn. Here, we explore 4 toy datasets:

* **Circles:** 2 concentric circles. We would like to see datapoints in each circle assigned to the same cluster
* **Two moons:** 2 half moons placed with one end into the opening of the other. We would like to see datapoints in each moon assigned to the same cluster.
* **Separated Blobs of datapoints:** We would like to see datapoints in each blob asssigned to the same cluster.
* **Separated Blobs of datapoints with different variance:** We would like to see datapoints in each blob asssigned to the same cluster

TIPS:
* some of the generated datasets have different number of clusters, be sure to take that into account when fitting the data
* For easier parameter selection, it's often a good idea to normalise the dataset, have a look at the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

In [ ]:
from sklearn import datasets
from itertools import cycle, islice

circles_points, circles_labels = datasets.make_circles(n_samples=1500, factor=0.5, noise=.05)
two_moons_points, two_moons_labels = datasets.make_moons(n_samples=1500, noise=.05)
blobs_points, blobs_labels = datasets.make_blobs(n_samples=n_samples, 
                                                 cluster_std=[.5, .5, .5],
                                                 random_state=200)
varied_points, varied_labels = datasets.make_blobs(n_samples=n_samples,
                                                   cluster_std=[1.0, 1.3, 0.5],
                                                   random_state=150)
def plot_clusters(X, labels=None, title=''):
    palette = ['green', 'crimson', 'gold', 'steelblue', 'orange']
    if labels is None:
        labels = [0] * X.shape[0]
        colors = np.array(['red'] * X.shape[0])
    else:
        colors = np.array(list(islice(cycle(palette), int(max(labels) + 1))))
    plt.figure()
    plt.scatter(X[:,0], X[:,1], s=10, color = colors[labels])
    plt.title(title)
    plt.show()

#visualise all the toy datasets
plot_clusters(circles_points, circles_labels, title='2 circles (generated data)')
plot_clusters(two_moons_points, two_moons_labels, title='2 moons (generated data)')
plot_clusters(blobs_points, blobs_labels, title='blobs (generated data)')
plot_clusters(varied_points, varied_labels, title='2 moons (generated data)')

### Exercise 3
Explore the impact of the different linkages in the 2 circles dataset and plot the results. Leave the linkage that appears to work the best as the first plot, followed by the remaining linkages to visualise the results.

For this case only, I provide a lot of the boilerplate code that will be useful for the next exercises.

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined above
# to visualise your results, you don't have to redefine it.
from sklearn.preprocessing import StandardScaler

ward = AgglomerativeClustering(n_clusters=2, linkage='ward')
complete = AgglomerativeClustering(n_clusters=2, linkage='complete')
average = AgglomerativeClustering(n_clusters=2, linkage='average')
single = AgglomerativeClustering(n_clusters=2, linkage='single')

algorithms = [ward, complete, average, single]
names = ['ward', 'complete', 'average', 'single']

# if you're going to normalise the data, do it here, if you don't think it's necessary, go right ahead.

for alg, name in zip(algorithms, names):
    # fit the algorithm with the data
    alg.fit(None) # replace None with the data
    plot_clusters(None) # replace None with the right parameters to make the plots

### Exercise 4
Explore the impact of the different linkages in the 2 moons dataset and plot the results. Leave the linkage that appears to work the best as the first plot, followed by the remaining linkages to visualise the results.

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined above
# to visualise your results, you don't have to redefine it.

### Exercise 5
Explore the impact of the different linkages in the separated blobs dataset and plot the results. Leave the linkage that appears to work the best as the first plot, followed by the remaining linkages to visualise the results.

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined above
# to visualise your results, you don't have to redefine it.

### Exercise 6
Explore the impact of the different linkages in the separated blobs with different variance dataset and plot the results. Leave the linkage that appears to work the best as the first plot, followed by the remaining linkages to visualise the results.

In [ ]:
# Use this cell to write your code. 
# Remember that you can use the plot_clusters function defined above
# to visualise your results, you don't have to redefine it.